In [1]:
import pandas as pd

In [3]:
import sys
sys.path.append("/Users/damoncrockett/differ/")

In [4]:
import differ

In [5]:
ft = pd.read_csv("170728_2015-01-01_alphasort.csv")

In [9]:
train_matrix = pd.read_csv("170728_2015-01-01_train-matrix-and-labels.csv")

In [12]:
tmp = train_matrix.drop(['outcome'],axis=1).apply(sum,axis=0) > 0
zcols = tmp.index[tmp==False]
train_matrix = train_matrix.drop(zcols,axis=1)

tmp = train_matrix.drop(['outcome'],axis=1)
ocols = [col for col in tmp.columns if len(tmp[col].value_counts())==1]
train_matrix = train_matrix.drop(ocols,axis=1)

In [13]:
dcols = train_matrix.drop(['outcome'],axis=1).columns
train_matrix[dcols] = train_matrix[dcols].applymap(float)

In [15]:
cols = train_matrix.columns
ft = ft[ft.feature.isin(cols)]
ft.reset_index(drop=True,inplace=True)

In [19]:
x = 10
ft.sort_values(by='feature_importance',ascending=False,inplace=True)
top_feats = ft.feature.iloc[:x]

/Library/Python/2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [21]:
cfeats = []
ifeats = []
featdists = []
subdists = []

nbins = 4 # for continuous features

for cfeat in top_feats:
    for i in ft.index:
        cfeats.append(cfeat)
        ifeat = ft.feature.loc[i]
        ifeats.append(ifeat)
        
        if ft.record_type.loc[i]=='ocnd':
            outputs = differ.KL_categorical(train_matrix,ifeat,cfeat)
        else:
            outputs = differ.KL_continuous(train_matrix,ifeat,cfeat,nbins)
        
        featdists.append(outputs[0])
        subdists.append(outputs[1])
        
df = pd.DataFrame({
        "candidate_feature":cfeats,
        "interacting_feature":ifeats,
        "feature_distance":featdists,
        "subset_distances":subdists
    })

In [22]:
df.to_csv("/Users/damoncrockett/Desktop/poster_interactions.csv",index=False)

In [29]:
cfeat = 'dispatch_id_p3m_dispatchtype_traffic_sum'

In [32]:
traff = df[df.candidate_feature==cfeat]

In [35]:
traff[traff.interacting_feature.str.contains("black")].to_csv("/Users/damoncrockett/Desktop/poster_interactions.csv",index=False)

In [30]:
df[df.candidate_feature==cfeat].to_csv("/Users/damoncrockett/Desktop/poster_interactions.csv",index=False)